In [3]:
import tensorflow as tf
import numpy as np
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
def createTrainMatrix(trainFile, wordsList, maxLen):
    chatDict = np.load(trainFile).item()
    cardinalDict = len(chatDict)
    xTrain = np.zeros((cardinalDict , maxLen), dtype='int32')
    yTrain = np.zeros((cardinalDict , maxLen), dtype='int32')
    
    for index,(comm, reply) in enumerate(chatDict.iteritems()):
        encoderMessage = np.full((maxlen), wordsList.index('<pad>'), dtype='int32')
        decoderMessage = np.full((maxlen), wordsList.index('<pad>'), dtype='int32')
        
        commList = comm.split()
        replyList = reply.split()
        commCount = len(commList)
        replyCount = len(replyList)
        
        if commCount > maxLen or replyCount >maxLen or commCount == 0 or replyCount==0:
            continue
        #convert comment and reply list string into numbers so tensorflow can work on it 
        
        for commIndex , word in enumerate(commList):
            try:
                encoderMessage[commIndex] = wordsList.index(word)
            except ValueError:   #if word is not contained in wordslist 
                encoderMessage[commIndex] = 0
        encoderMessage[commIndex+1] = wordsList.index('<EOS>')
        
        for replyIndex , word in enumerate(replyList):
            try:
                decoderMessage[replyIndex] = wordsList.index(word)
            except ValueError:   #if word is not contained in wordslist 
                decoderMessage[replyndex] = 0
        decoderMessage[replyIndex+1] = wordsList.index('<EOS>')
        
        xTrain[index] = encoderMessage
        yTrain[index] = decoderMessage
        
        xTrain = xTrain[~np.all(xTrain == 0 , axis = 1)]
        yTrain = yTrain[~np.all(yTrain == 0 , axis = 1)]
        cardinalDict = xTrain.shape[0]
        return cardinalDict, xTrain, yTrain
        
        
        

    
    

In [5]:
def getTrainingBatch(localXTrain, localYTrain, localBatchSize,maxLen ):
    num = randint(numTrainingExamples - localBatchSize -1)
    arr = localXTrain[num:num + localBatchSize]  #comments
    labels = localYTrain[num:num + localBatchSize]  #corresponding replies
    
    arr = list(arr)                        # reversing encoder string helps somewhat 
    for index, example in enumerate(arr):
        arr[index] = list(reversed(example))
    #lagged label for training input into decoder
    
    laggedLabels = []
    EOStokenindex = wordsList.index('EOS')
    padtokenindex = wordsList.index('pad')
    
    for label in labels:
        eosFound = np.argwhere(label == EOStokenindex)[0]
        shiftedLabel = np.roll(label,1)  # shifted label seems redundable check back later
        shiftedLabel[0] = EOStokenindex
        
        if (eosFound != maxLen -1):
            shiftedLabel[eosFound+1] = padtokenindex
        laggedLabels.append(shiftedLabel)
    # transpose them
    arr = np.asarray(arr).T.tolist()
    labels = labels.T.tolist()
    laggedLabels = np.asarray(laggedLabels).T.tolist()
    return arr ,labels ,laggedLabels

def translateToSentence(inputs, wordsList, encoder=False):
    EOStokenindex = wordsList.index('EOS')
    padtokenindex = wordsList.index('pad')
    numStrings = len(inputs[0])
    numLengthStrings = len(inputs)
    listOfStrings = ['']*numStrings
    
    for sentence in inputs:
        for index, wordint in sentence:
            if (wordint != EOStokenindex and wordint != padtokenindex):
                if(encoder):    #reverse encoded
                    listOfStrings[index] = wordsList[wordint]  + " " + listOfStrings[index]
                else :
                    listOfStrings[index] = listOfStrings[index] + wordsList[wordint]
    listOfStrings = (string.strip() for string in listOfStrings)
    return listOfStrings

def getTestInput(inputMessage, wordsList, maxLen):
    encoderMessage = np.full((maxLen), wordsList.index('<pad>'))
    messageList = inputMessage.lower().split()
    for index, word in messageList:
        try:
            encoderMessage[index] = wordsList.index(word)
        except ValueError:
            continue     #handle exception here in a good way
    encoderMessage[index+1] = wordsList.index('<EOS>')
    encoderMessage = encoderMessage[::-1]
    encoderMessageList =[]
    for wordint in encoderMessage:
        encoderMessageList.append(wordint)
    return encoderMessageList
    
    